# Tools

#### Models can request calls to tools that perform tasks such as fetching data from databases, searching web, or running code. Tools are pairings of:
1. A Schema, including the name, description and/or arguments definition (often a JSON schema)
2. A function or coroutine to execute the tool.

In [1]:
import os
from langchain.chat_models import init_chat_model

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

model = init_chat_model("groq:qwen/qwen3-32b")
response=model.invoke("Why do parrots talk?")
response.content

'<think>\nOkay, so I need to figure out why parrots talk. Let me start by recalling what I know about parrots. They\'re birds, right? Some species are known for mimicking human speech, like African Greys and Amazon parrots. But why do they do that?\n\nFirst, maybe it\'s related to their natural behavior. In the wild, parrots might use different sounds to communicate with each other. So maybe talking is an extension of that. But how does mimicking human speech fit into that?\n\nI remember reading that parrots have a special vocal organ called a syrinx, which allows them to produce a wide range of sounds. That might help them imitate human words. But why would they choose to mimic humans specifically?\n\nAnother thought: parrots are social animals. In the wild, they live in flocks and need to communicate with each other. If they\'re kept as pets, they might see humans as part of their "flock." So talking could be a way to interact and bond with their human companions. That makes sense be

In [2]:
from langchain.tools import tool

@tool
def get_weather(city: str) ->str:
    """ Get the weather for a city """
    return f"The weather in {city} is sunny."

model_with_tools=model.bind_tools([get_weather])


In [7]:
response=model_with_tools.invoke("What is the weather in Dallas?")

for tool_call in response.tool_calls:
    print(f"Tool: {tool_call['name']}\nArgs: {tool_call['args']}")

Tool: get_weather
Args: {'city': 'Dallas'}


#### Tools execution Loops

In [10]:
### Tools execution
# Step 1: Model generates tool calls

messages = [
    {"role": "user", "content": "What is the weather in Dallas?"}
]
ai_msg=model_with_tools.invoke(messages)
messages.append(ai_msg)
print(messages)
print("="*60)

# Step 2: Tool execution and collect results
for tool_call in ai_msg.tool_calls:
    # Execute tool with generated arguments
    print(tool_call)
    tool_result=get_weather.invoke(tool_call)
    messages.append(tool_result)
print(messages)
print("="*60)
# Step 3: Pass Input query, context back to model for final response
final_response=model_with_tools.invoke(messages)
print(final_response.text)
print("="*60)

[{'role': 'user', 'content': 'What is the weather in Dallas?'}, AIMessage(content='', additional_kwargs={'reasoning_content': "Okay, the user is asking for the weather in Dallas. Let me check the tools available. There's a function called get_weather that takes a city parameter. I need to call that function with the city set to Dallas. I'll make sure to format the tool call correctly within the XML tags.\n", 'tool_calls': [{'id': 'x2r4peqbt', 'function': {'arguments': '{"city":"Dallas"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 84, 'prompt_tokens': 153, 'total_tokens': 237, 'completion_time': 0.111647679, 'completion_tokens_details': {'reasoning_tokens': 60}, 'prompt_time': 0.006245717, 'prompt_tokens_details': None, 'queue_time': 0.050166373, 'total_time': 0.117893396}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model

In [11]:
messages

[{'role': 'user', 'content': 'What is the weather in Dallas?'},
 AIMessage(content='', additional_kwargs={'reasoning_content': "Okay, the user is asking for the weather in Dallas. Let me check the tools available. There's a function called get_weather that takes a city parameter. I need to call that function with the city set to Dallas. I'll make sure to format the tool call correctly within the XML tags.\n", 'tool_calls': [{'id': 'x2r4peqbt', 'function': {'arguments': '{"city":"Dallas"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 84, 'prompt_tokens': 153, 'total_tokens': 237, 'completion_time': 0.111647679, 'completion_tokens_details': {'reasoning_tokens': 60}, 'prompt_time': 0.006245717, 'prompt_tokens_details': None, 'queue_time': 0.050166373, 'total_time': 0.117893396}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'mode